In [1]:
import numpy as np
import pandas as pd
import nltk
import pyLDAvis
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
dataset = fetch_20newsgroups(remove=('headers','footers','quotes'))
documents = dataset.data[:1000]   

print("Total documents:", len(documents))


Total documents: 1000


In [6]:
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.9, min_df=5)
count_data = count_vectorizer.fit_transform(documents)


In [7]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=5)
tfidf_data = tfidf_vectorizer.fit_transform(documents)


In [8]:
lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_model.fit(count_data)

nmf_model = NMF(n_components=10, random_state=42)
nmf_model.fit(tfidf_data)


,n_components,10
,init,None
,solver,'cd'
,beta_loss,'frobenius'
,tol,0.0001
,max_iter,200
,random_state,42
,alpha_W,0.0
,alpha_H,'same'
,l1_ratio,0.0
,verbose,0


In [9]:
def display_topics(model, feature_names, no_top_words=10):
    topics_list = []
    for topic_idx, topic in enumerate(model.components_):
        topic_words = [feature_names[i] 
                       for i in topic.argsort()[:-no_top_words - 1:-1]]
        topics_list.append(topic_words)
        print(f"\nTopic {topic_idx+1}:")
        print(" ".join(topic_words))
    return topics_list

print("===== LDA Topics =====")
lda_topics = display_topics(lda_model, count_vectorizer.get_feature_names_out())

print("\n===== NMF Topics =====")
nmf_topics = display_topics(nmf_model, tfidf_vectorizer.get_feature_names_out())



===== LDA Topics =====

Topic 1:
max 145 04 tm p2 34 th ma 45 dos

Topic 2:
time just new car year think like greek good know

Topic 3:
jesus god people does know argument believe think true say

Topic 4:
armenian turkish armenians space nasa people genocide p2 government 000

Topic 5:
know don think moral help people use thanks morality problem

Topic 6:
don problem just know use card thanks does think team

Topic 7:
people like don just said right know gun left look

Topic 8:
windows like files program file pc data code microsoft using

Topic 9:
00 good 50 10 15 period 20 pp 11 excellent

Topic 10:
health use 1993 edu com information father son medical users

===== NMF Topics =====

Topic 1:
don think know just people like really way want good

Topic 2:
windows program files file ftp image run code pc window

Topic 3:
armenian armenians turkish genocide soviet people government armenia turks russian

Topic 4:
god truth believe christian moral absolute faith bible know belief

Topic 5

In [10]:
print("LDA Perplexity:", lda_model.perplexity(count_data))


LDA Perplexity: 1750.8273336863276


In [11]:
texts = [doc.split() for doc in documents]
dictionary = Dictionary(texts)

coherence_lda = CoherenceModel(
    topics=lda_topics,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)

print("LDA Coherence Score:", coherence_lda.get_coherence())

coherence_nmf = CoherenceModel(
    topics=nmf_topics,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)

print("NMF Coherence Score:", coherence_nmf.get_coherence())


LDA Coherence Score: 0.3731797603822794
NMF Coherence Score: 0.4443292572695654


In [13]:
import pyLDAvis

pyLDAvis.enable_notebook()

# Topic-term distribution
topic_term_dists = lda_model.components_ / lda_model.components_.sum(axis=1)[:, None]

# Document-topic distribution
doc_topic_dists = lda_model.transform(count_data)

# Document lengths
doc_lengths = count_data.sum(axis=1).A1

# Vocabulary
vocab = count_vectorizer.get_feature_names_out()

# Term frequencies
term_frequency = count_data.sum(axis=0).A1

lda_vis = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency
)

lda_vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.129859  0.024854       1        1  22.424107
3      0.050145 -0.190063       2        1  12.167637
1     -0.092678  0.075855       3        1  12.031739
5     -0.087060  0.086189       4        1  11.795566
6     -0.116760 -0.047086       5        1  11.662697
9      0.060318 -0.175256       6        1   7.716420
8      0.293050  0.011610       7        1   6.448589
4     -0.111625  0.009188       8        1   6.147481
7     -0.041869  0.028000       9        1   5.732194
0      0.176339  0.176709      10        1   3.873570, topic_info=         Term        Freq       Total Category  logprob  loglift
1602      max  319.000000  319.000000  Default  30.0000  30.0000
30        145   98.000000   98.000000  Default  29.0000  29.0000
0          00  106.000000  106.000000  Default  28.0000  28.0000
1397    jesus  188.000000  188.000000  Default  27.0000  27.0000
2744  windows  108.000000  108.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
744       day   11.950194   99.097940  Topic10  -5.2213   1.1356
27         14   11.558652   87.646221  Topic10  -5.2546   1.2251
2554     time   11.542359  233.475518  Topic10  -5.2560   0.2439
63         25    9.451912   68.630418  Topic10  -5.4559   1.2685
1246       hi    8.962338   37.281909  Topic10  -5.5090   1.8255

[641 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         5  0.009421     00
0         7  0.979825     00
1         1  0.066706    000
1         2  0.767119    000
1         3  0.133412    000
...     ...       ...    ...
2783      4  0.070369  years
2783      5  0.101644  years
2783      6  0.164194  years
2783      8  0.039094  years
2783      9  0.039094  years

[1866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 6, 7, 10, 9, 5, 8, 1])